<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Load datasets


In [2]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [3]:
# Dados Dengue

merge_dengue = dengue.append([dengue19,dengue18,dengue17,dengue16], ignore_index = True)
merge_dengue = merge_dengue[["febre", "mialgia",'vomito', "dt_notificacao",'nu_idade','nausea','cefaleia','dor_costas','diabetes','renal']]
merge_dengue['dt_notificacao'] = pd.to_datetime(merge_dengue['dt_notificacao'])

# Agrupamos por número de casos por dia

#merge_dengue = merge_dengue.groupby([pd.Grouper(key="dt_notificacao", freq="D")]).size().reset_index(name="CASOS_DENGUE")


# Dados Emlurb

merge_emlurb = emlurb.append([emlurb19,emlurb18,emlurb17,emlurb16], ignore_index = True)
merge_emlurb = merge_emlurb[["GRUPOSERVICO_CODIGO", "DATA_DEMANDA"]]
merge_emlurb = merge_emlurb[(merge_emlurb.GRUPOSERVICO_CODIGO == 11)]
merge_emlurb["DATA_DEMANDA"] = pd.to_datetime(merge_emlurb["DATA_DEMANDA"],errors='coerce')

# Agrupamos por número de solicitações por dia

#merge_emlurb = merge_emlurb.groupby([pd.Grouper(key="DATA_DEMANDA", freq="D")]).size().reset_index(name="CHAMAD0S_EMLURB")

#year_filter = merge_emlurb[(merge_emlurb['DATA_DEMANDA'].dt.year == 2018)]


merge_dengue = merge_dengue.dropna()
merge_dengue['febre'] = merge_dengue['febre'].astype('int')
merge_dengue['mialgia'] = merge_dengue['mialgia'].astype('int')
merge_dengue['vomito'] = merge_dengue['vomito'].astype('int')
merge_dengue['nausea'] = merge_dengue['nausea'].astype('int')
merge_dengue['cefaleia'] = merge_dengue['cefaleia'].astype('int')
merge_dengue['dor_costas'] = merge_dengue['dor_costas'].astype('int')
merge_dengue['diabetes'] = merge_dengue['diabetes'].astype('int')
merge_dengue['renal'] = merge_dengue['renal'].astype('int')
merge_dengue

,febre,mialgia,vomito,dt_notificacao,nu_idade,nausea,cefaleia,dor_costas,diabetes,renal
0,1,1,2,2020-06-26,2021,2,1,2,2,2
1,2,2,2,2020-06-22,2001,2,2,2,2,2
2,1,1,1,2020-01-09,4029,1,1,2,2,2
3,1,1,2,2020-01-03,4039,2,2,2,2,2
4,2,1,1,2020-01-13,4062,1,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
32059,1,1,2,2016-12-29,4019,2,1,2,2,2
32060,1,2,2,2017-01-09,4031,2,1,2,2,2
32061,1,1,2,2017-01-02,4016,2,1,2,2,2
32062,1,2,2,2016-12-27,2010,2,2,2,2,2


## Feature de predição

1.   Número de casos de dengue por dia.
2.   Número de solicitações de drenagem por dia.



## Separação de dados: Treino, Validação e Teste

Texto.

In [4]:
def split_values(df, column_name):
  X = df.drop(columns=[column_name,'dt_notificacao']).values
  y = df[column_name].values

  return X, y

In [26]:
features_results = merge_dengue.drop(columns=['febre','dt_notificacao']).values
labels_results = merge_dengue['febre'].values

feature_train, feature_test, label_train, label_test = train_test_split(features_results, labels_results, test_size=0.2,random_state=1)
feature_train, feature_validation, label_train, label_validation =  train_test_split(feature_train, label_train, test_size=0.25,random_state=1)

In [21]:
train, validation, test = np.split(merge_dengue.sample(frac=1, random_state=42), [int(.6*len(merge_dengue)), int(.8*len(merge_dengue))])

column_name = 'febre'

X_train, y_train = split_values(train, column_name)
X_valid, y_valid = split_values(validation, column_name)
X_test,  y_test  = split_values(test, column_name)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(15547, 8)
(5183, 8)
(5183, 8)


## Escolha de Algoritmos

*   Decision Tree
*   Random Forest
*   Naive Bayes
*   MLP
*   KNN



In [29]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
naive_bayes   = GaussianNB()
ml_perceptron = MLPClassifier()

decision_tree.fit(X_train, y_train)
y_decision_tree_train = decision_tree.predict(X_train)
y_decision_tree_test  = decision_tree.predict(X_test)

random_forest.fit(X_train, y_train)
y_random_forest_train = random_forest.predict(X_train)
y_random_forest_test  = random_forest.predict(X_test)

naive_bayes.fit(X_train, y_train)
y_naive_bayes_train = naive_bayes.predict(X_train)
y_naive_bayes_test  = naive_bayes.predict(X_test)

ml_perceptron.fit(X_train, y_train)
y_ml_perceptron_train = ml_perceptron.predict(X_train)
y_ml_perceptron_test  = ml_perceptron.predict(X_test)

In [33]:
print("=== Precisão Testes === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_test, y_decision_tree_test))
print("Precisão Random Forest:", accuracy_score(y_test, y_random_forest_test))
print("Precisão Naive Bayes:" , accuracy_score(y_test, y_naive_bayes_test))
print("Precisão MLP: ", accuracy_score(y_test, y_ml_perceptron_test))

print("\n\n=== Precisão Treino === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_train, y_decision_tree_train))
print("Precisão Random Forest:", accuracy_score(y_train, y_random_forest_train))
print("Precisão Naive Bayes:" , accuracy_score(y_train, y_naive_bayes_train))
print("Precisão MLP: ", accuracy_score(y_train, y_ml_perceptron_train))

=== Precisão Testes === 


Precisão Decision Tree: 0.8817287285355971
Precisão Random Forest: 0.880764036272429
Precisão Naive Bayes: 0.729500289407679
Precisão MLP:  0.8126567624927649


=== Precisão Treino === 


Precisão Decision Tree: 0.8882099440406509
Precisão Random Forest: 0.8882099440406509
Precisão Naive Bayes: 0.7263137582813405
Precisão MLP:  0.8135974786132373


## MLFlow e Hiper-parâmetros

Nessa etapa, adicionaremos o MLFlow e o Optuna para seleção de hiper-parâmetros sobre o conjunto de validação.

In [36]:
!pip install mlflow --quiet
!pip install optuna --quiet

import mlflow
import optuna

### Decision Tree

In [ ]:
def objective_decisionTree(trial):
    max_depth = trial.suggest_int("max_depth", 1, 15, 1)
    split     = trial.suggest_int('min_samples_split', 3, 15, 1)
    leaf      = trial.suggest_int('min_samples_leaf', 3, 15, 1)

    decisionTree_selection = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=leaf, min_samples_split=split)
    decisionTree_selection.fit(X_train, y_train)
    
    prediction_validation = decisionTree_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_decisionTree, n_trials=50)
print(study_dt.best_trial)

In [ ]:
print(study_dt.best_trial.value)
print(study_dt.best_trial.params)

### Random Forest

In [ ]:
def objective_randomForest(trial):
    forrest_n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    forrest_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    forrest_max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    randomForrest_selection = RandomForestClassifier(n_estimators=forrest_n_estimators, criterion=forrest_criterion, max_features=forrest_max_features)
    randomForrest_selection.fit(X_train, y_train) 

    prediction_validation = randomForrest_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_randomForest, n_trials=50)
print(study_rf.best_trial)

In [45]:
print(study_rf.best_trial.value)
print(study_rf.best_trial.params)

0.8819216669882307
{'n_estimators': 501, 'criterion': 'gini', 'max_features': 'sqrt'}


### Naive Bayes

In [ ]:
def objective_naiveBayes(trial):
    nb_var_smoothing = trial.suggest_float('var_smoothing', 1e-9, 1e9, log=True)

    naiveBayes_selection = GaussianNB(var_smoothing=nb_var_smoothing)
    naiveBayes_selection.fit(X_train, y_train)     

    prediction_validation = naiveBayes_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_nb = optuna.create_study(direction="maximize")
study_nb.optimize(objective_naiveBayes, n_trials=50)
print(study_nb.best_trial)

In [50]:
print(study_nb.best_trial.value)
print(study_nb.best_trial.params)

0.8730464981670847
{'var_smoothing': 1.941655811893446e-07}


### MLP

In [ ]:
def objective_MLP(trial):

    #mlp_selection = MLPClassifier()
    #mlp_selection.fit(X_train, y_train)     

    #prediction_validation = mlp_selection.predict(X_valid)
    #accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_mlp = optuna.create_study(direction="maximize")
study_mlp.optimize(objective_MLP, n_trials=50)
print(study_mlp.best_trial)

In [ ]:
print(study_mlp.best_trial.value)
print(study_mlp.best_trial.params)

## Diagnóstico de melhor modelo

Após os testes realizados com o Random Forest, Decision Tree, Naive Bayes e MLP, encontramos que o algoritmo que melhor resultado obeteve foi blabla.


In [51]:
## heatmap

## Feature importance

Verificamos o nível de relevância das colunas do dataset.

# **MLP**

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics, activations

resolver = tensorflow.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tensorflow.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tensorflow.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tensorflow.config.list_logical_devices('TPU'))

In [ ]:
def MLP():
    nu_shape = input_shape[0] * input_shape[1]
    curve = activations.sigmoid
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape),
            layers.Reshape((nu_shape,)),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(nu_shape, activation=curve),
            layers.Dropout(0.1),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(num_classes, activation=activations.softmax)
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(learning_rate=1), metrics=["accuracy", metrics.Recall(), metrics.Precision()], steps_per_execution=50)

    return model

In [ ]:
def LSTM(hidden_layer_size, look_back = 7):
  model = Sequential()
  model.add(LSTM(hidden_layer_size, input_shape=(look_back,1)))
  model.add(Dense(1)) # to have a regressor
  model.compile(loss='mean_squared_error', optimizer='adam', steps_per_execution=50)
  return model